<a href="https://colab.research.google.com/github/worldofaryavart/colab_notebooks/blob/colabnotebook/making_scraperModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install transformers sentence-transformers torch scikit-learn datasets spacy wordnet nltk

# Download spaCy's language model
!python -m spacy download en_core_web_sm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
  Created wheel for wordnet: filename=wordnet-0.0.1b2-py3-none-any.whl size=10498 sha256=f1cf9c2158df60782dc68ec1b59bc92c85bd7d136c80934b872ae3a006a22bb1
  Stored in directory: /root/.cache/pip/wheels/c0/a1/e8/4649c8712033dcdbd1e64a0fc75216a5d1769665852c36b4f9
Successfully built wordnet
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may nee

In [ ]:
# Import necessary libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd

In [ ]:
# Load Quora Question Pairs dataset
dataset = load_dataset("quora", split="train")

# Display some samples
print(dataset[:1])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

quora.py:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

The repository for quora contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/quora.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

{'questions': [{'id': [1, 2], 'text': ['What is the step by step guide to invest in share market in india?', 'What is the step by step guide to invest in share market?']}], 'is_duplicate': [False]}


In [ ]:
dataset_df = dataset.to_pandas().head(100)
print(f"using {dataset_df.shape[0]} examples for fine-tuning.")
print(dataset_df.head())
print(dataset_df.info())


using 100 examples for fine-tuning.
                                           questions  is_duplicate
0  {'id': [1, 2], 'text': ['What is the step by s...         False
1  {'id': [3, 4], 'text': ['What is the story of ...         False
2  {'id': [5, 6], 'text': ['How can I increase th...         False
3  {'id': [7, 8], 'text': ['Why am I mentally ver...         False
4  {'id': [9, 10], 'text': ['Which one dissolve i...         False
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   questions     100 non-null    object
 1   is_duplicate  100 non-null    bool  
dtypes: bool(1), object(1)
memory usage: 1.0+ KB
None


In [ ]:
dataset_df['questions'].iloc[0]

{'id': array([1, 2], dtype=int32),
 'text': array(['What is the step by step guide to invest in share market in india?',
        'What is the step by step guide to invest in share market?'],
       dtype=object)}

In [ ]:
train_data, val_data = train_test_split(dataset_df, test_size=0.2, random_state = 42)

train_data = Dataset.from_pandas(train_data)
val_data = Dataset.from_pandas(val_data)

print(train_data[0])
# print(val_data)

{'questions': {'id': [111, 112], 'text': ['How difficult is it get into RSI?', "Do you apply for programs like RSI when you're a rising senior?"]}, 'is_duplicate': False, '__index_level_0__': 55}


In [ ]:
print(train_data['questions'][0]['text'])

['What are some of the short stories?', 'What are some great short stories with a twist ending?']


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(batch):
    return tokenizer([q['text'] for q in batch['questions']],
                     padding="max_length", truncation=True, max_length=128)

# Tokenize train and validation data
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

# Set the format for PyTorch
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'is_duplicate'])
val_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'is_duplicate'])

print(train_data[0])  # Check one tokenized sample


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

{'is_duplicate': tensor(False), 'input_ids': tensor([  101,  2129,  3697,  2003,  2009,  2131,  2046, 12667,  2072,  1029,
          102,  2079,  2017,  6611,  2005,  3454,  2066, 12667,  2072,  2043,
         2017,  1005,  2128,  1037,  4803,  3026,  1029,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     

In [ ]:
print(train_data.shape)

(80, 6)


In [ ]:
print(train_data[0])

{'is_duplicate': tensor(False), 'input_ids': tensor([  101,  2129,  3697,  2003,  2009,  2131,  2046, 12667,  2072,  1029,
          102,  2079,  2017,  6611,  2005,  3454,  2066, 12667,  2072,  2043,
         2017,  1005,  2128,  1037,  4803,  3026,  1029,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     

In [ ]:
# print(train_data.features)
print(train_data[0]['is_duplicate'])
# print(train_data['questions'])
print(train_data[0]['input_ids'])
print(train_data[0]['attention_mask'])


tensor(False)
tensor([  101,  2129,  3697,  2003,  2009,  2131,  2046, 12667,  2072,  1029,
          102,  2079,  2017,  6611,  2005,  3454,  2066, 12667,  2072,  2043,
         2017,  1005,  2128,  1037,  4803,  3026,  1029,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [ ]:
# Load a pre-trained BERT model for binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.nn import CrossEntropyLoss

# Define the loss function
loss_fn = CrossEntropyLoss()


In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

In [ ]:
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8)

optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from torch.nn import functional as F

epochs = 1

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['is_duplicate'].to(device)

      optimizer.zero_grad()
      labels = labels.type(torch.LongTensor).to(device)

      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs.loss
      total_loss += loss.item()

      loss.backward()
      optimizer.step()

      print(f"Batch Loss: {loss.item()}")

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

print("Training complete!")

NameError: name 'model' is not defined